In [ ]:
%%writefile parallel_dfs_openmp.cpp
#include <iostream>
#include <stack>
#include <vector>
#include <omp.h>

using namespace std;

const int MAX = 1000;
int graph[MAX][MAX], visited[MAX];

void parallel_dfs(int start, int n) {
    stack<int> s;
    s.push(start);

    #pragma omp parallel
    {
        while (!s.empty()) {
            int curr;

            #pragma omp critical
            {
                if (!s.empty()) {
                    curr = s.top();
                    s.pop();
                } else {
                    curr = -1;
                }
            }

            if (curr != -1 && !visited[curr]) {
                visited[curr] = 1;

                #pragma omp parallel for
                for (int i = 0; i < n; i++) {
                    if (graph[curr][i] && !visited[i]) {
                        #pragma omp critical
                        {
                            if (!visited[i]) {
                                s.push(i);
                            }
                        }
                    }
                }
            }
        }
    }
}

int main() {
    int n, start;
    cout << "Enter number of vertices: ";
    cin >> n;

    cout << "Enter adjacency matrix:\n";
    for (int i = 0; i < n; i++)
        for (int j = 0; j < n; j++)
            cin >> graph[i][j];

    cout << "Enter starting vertex: ";
    cin >> start;

    parallel_dfs(start, n);

    cout << "DFS traversal: ";
    for (int i = 0; i < n; i++)
        if (visited[i])
            cout << i << " ";
    cout << endl;

    return 0;
}


Writing parallel_dfs_openmp.cpp


In [ ]:
!g++ -fopenmp parallel_dfs_openmp.cpp -o parallel_dfs_openmp

In [ ]:
!./parallel_dfs_openmp

Enter number of vertices: 3
Enter adjacency matrix:
1 1 1 
0 1 1 
1 0 0 
Enter starting vertex: 0
DFS traversal: 0 1 2 
